# Title
[]()

In [2]:

import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from silvhua import *
# from datetime import datetime
# sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")

In [3]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', 100)
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# from `main.py`

In [4]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
from prompts import * # .py file stored in the path above
from db_orm import *
from sources import *
from orm_summarize import *

#########
# Prep: Set parameters
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=True
folder_path = '/database'


def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py

    if sources_df['title'].tolist() == references_df_dict[iteration_id]['title'].tolist():

        # ##### 
        # Step 3: Create summaries (functions contained in orm_summarize.py)
        chatbot_dict = batch_summarize( # orm_summarize.py
            sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
            simplify_task, simplify_audience, format_task,
            chatbot_dict, temperature=temperature,
            system_role=system_role, model=model, max_tokens=1000,
            n_choices=n_choices, pause_per_request=pause_per_request,
            iteration_id=iteration_id, save_outputs=save_outputs
            )
        #########
        # Step 4: Create summaries table
        qna_dict = create_summaries_df(
            qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
            )

        ##########
        # Step 5: Add results to summaries and prompts table 
        bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

        return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, article_limit=article_limit)
    print(qna_dict[iteration_id])

# Update `main.py` so it can use local article files or articles in gpt_queue table 

In [3]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
# Prep: Set parameters
folder_path = '../text/2023-07-19 discussion'
section = 'discussion'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py

    if sources_df['title'].tolist() == references_df_dict[iteration_id]['title'].tolist():

        # ##### 
        # Step 3: Create summaries (functions contained in orm_summarize.py)
        chatbot_dict = batch_summarize( # orm_summarize.py
            sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
            simplify_task, simplify_audience, format_task,
            chatbot_dict, temperature=temperature,
            system_role=system_role, model=model, max_tokens=1000,
            n_choices=n_choices, pause_per_request=pause_per_request,
            iteration_id=iteration_id, save_outputs=save_outputs
            )
        #########
        # Step 4: Create summaries table
        qna_dict = create_summaries_df(
            qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
            )

        ##########
        # Step 5: Add results to summaries and prompts table 
        bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

        return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9])

Adding 9 rows to the database...
	Daily Energy Expenditure through the Human Life Course
	Differential effects of attentional focus strategies during long-term resistance training
	Effect of different doses of supervised exercise on food intake, metabolism, and non-exercise physical activity: The E-MECHANIC randomized controlled trial.
	Habitual tub bathing and risks of incident coronary heart disease and stroke
	High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.
	Higher muscle fiber conduction velocity and early rate of torque development in chronically strength-trained individuals
	Impact of insufficient sleep on dysregulated blood glucose control under standardised meal conditions
	Recovery From Eccentric Squat Exercise in Resistance-Trained Young and Master Athletes With Similar Maximum Strength: Combining Cold Water Immersion and Compressi

In [2]:
qna_dict

,title,body,section,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,text
0,Daily Energy Expenditure through the Human Lif...,In addition to providing empirical measures an...,discussion,"Total daily energy expenditure (""total expendi...","Science (New York, N.Y.)","Herman Pontzer, Yosuke Yamada, Hiroyuki Sagaya...",2021,Aug,373,6556,808,812,10.1126/science.abe5017,In addition to providing empirical measures an...
1,Differential effects of attentional focus stra...,This is the first study to investigate the eff...,discussion,The purpose of this study was to investigate t...,European journal of sport science,"Brad Jon Schoenfeld, Andrew Vigotsky, Bret Con...",2018,,18,5,705,712,10.1080/17461391.2018.1447020,This is the first study to investigate the eff...
2,Effect of different doses of supervised exerci...,In this large randomized controlled trial with...,discussion,"Exercise is recommended for weight management,...",The American journal of clinical nutrition,"Corby K Martin, William D Johnson, Candice A M...",2019,,110,3,583,592,10.1093/ajcn/nqz054,In this large randomized controlled trial with...
3,Habitual tub bathing and risks of incident cor...,,discussion,Tub bathing is considered to have a preventive...,Heart (British Cardiac Society),"Tomohiko Ukai, Hiroyasu Iso, Kazumasa Yamagish...",2020,,106,10,732,737,10.1136/heartjnl-2019-315752,
4,High doses of anti-inflammatory drugs compromi...,The interest in this study was spurred by the ...,discussion,This study tested the hypothesis that high dos...,"Acta physiologica (Oxford, England)","M Lilja, M Mandi&#x107;, W Apr&#xf3;, M Melin,...",2018,,222,2,,,10.1111/apha.12948,The interest in this study was spurred by the ...
5,Higher muscle fiber conduction velocity and ea...,MFCV was measured during explosive force contr...,discussion,Strength-trained individuals (ST) develop grea...,"Journal of applied physiology (Bethesda, Md. :...","A Del Vecchio, F Negro, D Falla, I Bazzucchi, ...",2018,,125,4,1218,1226,10.1152/japplphysiol.00025.2018,MFCV was measured during explosive force contr...
6,Impact of insufficient sleep on dysregulated b...,"Here, we describe for the first time how sleep...",discussion,"Sleep, diet and exercise are fundamental to me...",Diabetologia,"Neli Tsereteli, Raphael Vallat, Juan Fernandez...",2022,,65,2,356,365,10.1007/s00125-021-05608-y,"Here, we describe for the first time how sleep..."
7,Recovery From Eccentric Squat Exercise in Resi...,The study aimed at investigating the effects o...,discussion,The aim of this study was to investigate wheth...,Frontiers in physiology,"Julian Schmidt, Alexander Ferrauti, Michael Ke...",2021,,12,,665204,,10.3389/fphys.2021.665204,The study aimed at investigating the effects o...
8,The Impact of Sleep Duration on Performance Am...,"This literature review, although diverse in th...",discussion,"The athletic advantage of sleep, although comm...",Clinical journal of sport medicine : official ...,"Gregory W Kirschen, Jason J Jones, Lauren Hale",2020,,30,5,503,512,10.1097/JSM.0000000000000622,"This literature review, although diverse in th..."


In [5]:
qna_dict[iteration_id]

TypeError: 'NoneType' object is not subscriptable

# Iteration 2

In [6]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
# Prep: Set parameters
folder_path = '../text/2023-07-19 discussion'
section = 'discussion'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9])

Adding 9 rows to the database...
	** Already exists in the database: Daily Energy Expenditure through the Human Life Course.
	** Already exists in the database: Differential effects of attentional focus strategies during long-term resistance training.
	** Already exists in the database: Effect of different doses of supervised exercise on food intake, metabolism, and non-exercise physical activity: The E-MECHANIC randomized controlled trial..
	** Already exists in the database: Habitual tub bathing and risks of incident coronary heart disease and stroke.
	** Already exists in the database: High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults..
	** Already exists in the database: Higher muscle fiber conduction velocity and early rate of torque development in chronically strength-trained individuals.
	** Already exists in the database: Impact of insu

# iteration 3

In [1]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
# Prep: Set parameters
folder_path = '../text/2023-07-19 discussion'
section = 'discussion'
local = False
n_choices = 1
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9])

Adding 9 rows to the database...
	** Already exists in the database: Daily Energy Expenditure through the Human Life Course.
	** Already exists in the database: Differential effects of attentional focus strategies during long-term resistance training.
	** Already exists in the database: Effect of different doses of supervised exercise on food intake, metabolism, and non-exercise physical activity: The E-MECHANIC randomized controlled trial..
	** Already exists in the database: Habitual tub bathing and risks of incident coronary heart disease and stroke.
	** Already exists in the database: High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults..
	** Already exists in the database: Higher muscle fiber conduction velocity and early rate of torque development in chronically strength-trained individuals.
	** Already exists in the database: Impact of insu

# Iteration 4 from `2023-07-14 full` folder

In [2]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

Flagged: 
	Article title: Exploring participants perspectives on adverse events due to resistance training: a qualitative study
	PubMed title: Exploring participants' perspectives on adverse events due to resistance training: a qualitative study.

Adding 14 rows to the database...
	A systematic review examining associations between physical activity, sedentary behaviour, and sleep duration with quality of life in older adults aged 65 years and above
	Advertising expenditures across media on food and beverage products heavily advertised on youth-appealing television stations in Canada
	Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output
	Creatine supplementation combined with blood flow restriction training enhances muscle thickness and performance: a randomized, placebo-controlled, and double-blind study
	Economic burden of low muscle strength in Canadian adults
	Ex

# Batch 5

In [3]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care
	Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity
	Effects of exercise timing on metabolic health
	The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review
New records added successfully (if applicable)!
Query: SELECT * from sources ORDER BY id DESC LIMIT 4
**Text #191 prompt #1 of 2**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 89 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\orm_summarize.py : Th

# Iteration 6: use the correct articles from the queue

In [12]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, limit, order_by='id', order='ASC'):
    """
    Return a database table as a pandas dataframe.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).order_by(order_by).limit(limit).all()
        return result
        # return pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        # return pd.Series({column.name: getattr(row, column.name) for column in row.__table__.columns})

    sources_df = input_df.apply(row_to_dict, axis=1)
    # sources_df = session.query(Sources).filter_by(
    #     # title=input_df['title'].tolist()[0],
    #     # section=input_df['section'].tolist()[0],
    #     # title=input_df.loc[0, 'title'],
    #     # section=input_df.loc[0, 'section'],
    # # ).order_by(order_by).limit(limit).all()

    # ).order_by(order_by).limit(limit).all()
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    sources_df = get_from_queue(input_df=text_df, limit=article_limit, order_by='id', order='ASC')
    return sources_df

    # # ##### 
    # # Step 3: Get the new sources for summarization
    # sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # # ##### 
    # # Step 4: Create summaries (functions contained in orm_summarize.py)
    # chatbot_dict = batch_summarize( # orm_summarize.py
    #     sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
    #     simplify_task, simplify_audience, format_task,
    #     chatbot_dict, temperature=temperature,
    #     system_role=system_role, model=model, max_tokens=1000,
    #     n_choices=n_choices, pause_per_request=pause_per_request,
    #     iteration_id=iteration_id, save_outputs=save_outputs
    #     )
    # #########
    # # Step 5: Create summaries table
    # qna_dict = create_summaries_df(
    #     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
    #     )

    # ##########
    # # Step 5: Add results to summaries and prompts table 
    # bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    # return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!


In [13]:
qna_dict

0    [<db_orm.Sources object at 0x000001DE8347C7D0>]
1    [<db_orm.Sources object at 0x000001DE8347FB50>]
2    [<db_orm.Sources object at 0x000001DE8347F610>]
3    [<db_orm.Sources object at 0x000001DE8347D3D0>]
dtype: object

In [16]:
row = qna_dict.loc[0][0]
pd.Series({column.name: getattr(row, column.name) for column in row.__table__.columns})

id                                                           188
title          A systematic review of patient barriers and fa...
text           Despite increasing awareness of the consequenc...
abstract       Numerous barriers are experienced by people wi...
publication    Obesity reviews : an official journal of the I...
authors        Maxim de Jong, N&#xfa;ria Jansen, Marienke van...
year                                                        2023
month                                                        Aug
pub_volume                                                    24
pub_issue                                                      8
start_page                                                e13571
end_page                                                        
doi                                            10.1111/obr.13571
section                                               discussion
dtype: object

In [10]:
for index, row in enumerate(qna_dict[1]):
    print(pd.Series({column.name: getattr(row, column.name) for column in row.__table__.columns}))

id                                                            32
title          A systematic review of patient barriers and fa...
text           Despite increasing awareness of the consequenc...
abstract       Numerous barriers are experienced by people wi...
publication    Obesity reviews : an official journal of the I...
authors        Maxim de Jong, N&#xfa;ria Jansen, Marienke van...
year                                                        2023
month                                                        Aug
pub_volume                                                    24
pub_issue                                                      8
start_page                                                e13571
end_page                                                        
doi                                            10.1111/obr.13571
section                                                      NaN
dtype: object


## 6.1

In [21]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, limit, order_by='id', order='ASC'):
    """
    Return a database table as a pandas dataframe.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).order_by(order_by).limit(1).all()[0]
        # return result
        return pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        # return pd.Series({column.name: getattr(row, column.name) for column in row.__table__.columns})

    sources_df = input_df.apply(row_to_dict, axis=1)
    # sources_df = session.query(Sources).filter_by(
    #     # title=input_df['title'].tolist()[0],
    #     # section=input_df['section'].tolist()[0],
    #     # title=input_df.loc[0, 'title'],
    #     # section=input_df.loc[0, 'section'],
    # # ).order_by(order_by).limit(limit).all()

    # ).order_by(order_by).limit(limit).all()
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    sources_df = get_from_queue(input_df=text_df, limit=article_limit, order_by='id', order='ASC')
    return sources_df

    # # ##### 
    # # Step 3: Get the new sources for summarization
    # sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # # ##### 
    # # Step 4: Create summaries (functions contained in orm_summarize.py)
    # chatbot_dict = batch_summarize( # orm_summarize.py
    #     sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
    #     simplify_task, simplify_audience, format_task,
    #     chatbot_dict, temperature=temperature,
    #     system_role=system_role, model=model, max_tokens=1000,
    #     n_choices=n_choices, pause_per_request=pause_per_request,
    #     iteration_id=iteration_id, save_outputs=save_outputs
    #     )
    # #########
    # # Step 5: Create summaries table
    # qna_dict = create_summaries_df(
    #     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
    #     )

    # ##########
    # # Step 5: Add results to summaries and prompts table 
    # bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    # return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!


In [22]:
qna_dict

,id,title,text,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,section
0,188,A systematic review of patient barriers and fa...,Despite increasing awareness of the consequenc...,Numerous barriers are experienced by people wi...,Obesity reviews : an official journal of the I...,"Maxim de Jong, N&#xfa;ria Jansen, Marienke van...",2023,Aug,24,8,e13571,,10.1111/obr.13571,discussion
1,189,Dietary pulses as a means to improve the gut m...,"According to the World Health Organization, th...",A dysbiotic intestinal microbiome has been lin...,Obesity reviews : an official journal of the I...,"Hannah St John, &#xc9;ric Doucet, Krista A Power",2023,,,,e13598,,10.1111/obr.13598,discussion
2,190,Effects of exercise timing on metabolic health,"In the last decades, the dramatic increase in ...",The increasing prevalence of metabolic syndrom...,Obesity reviews : an official journal of the I...,"Jos&#xe9; Ignacio Mart&#xed;nez-Montoro, Javie...",2023,,,,e13599,,10.1111/obr.13599,discussion
3,191,The burden of type 2 diabetes mellitus in stat...,Type 2 diabetes mellitus (T2D) is a complex me...,Type 2 diabetes mellitus (T2D) is a highly pre...,Obesity reviews : an official journal of the I...,"Carlos Alexandre Soares Andrade, Balqees Shahi...",2023,,,,e13593,,10.1111/obr.13593,discussion


## 6.2

In [32]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')
    return sources_df

    # # ##### 
    # # Step 3: Get the new sources for summarization
    # sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # # ##### 
    # # Step 4: Create summaries (functions contained in orm_summarize.py)
    # chatbot_dict = batch_summarize( # orm_summarize.py
    #     sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
    #     simplify_task, simplify_audience, format_task,
    #     chatbot_dict, temperature=temperature,
    #     system_role=system_role, model=model, max_tokens=1000,
    #     n_choices=n_choices, pause_per_request=pause_per_request,
    #     iteration_id=iteration_id, save_outputs=save_outputs
    #     )
    # #########
    # # Step 5: Create summaries table
    # qna_dict = create_summaries_df(
    #     qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
    #     )

    # ##########
    # # Step 5: Add results to summaries and prompts table 
    # bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    # return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!


In [34]:
qna_dict

,id,title,text,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,section
0,188,A systematic review of patient barriers and fa...,Despite increasing awareness of the consequenc...,Numerous barriers are experienced by people wi...,Obesity reviews : an official journal of the I...,"Maxim de Jong, N&#xfa;ria Jansen, Marienke van...",2023,Aug,24,8,e13571,,10.1111/obr.13571,discussion
1,189,Dietary pulses as a means to improve the gut m...,"According to the World Health Organization, th...",A dysbiotic intestinal microbiome has been lin...,Obesity reviews : an official journal of the I...,"Hannah St John, &#xc9;ric Doucet, Krista A Power",2023,,,,e13598,,10.1111/obr.13598,discussion
2,190,Effects of exercise timing on metabolic health,"In the last decades, the dramatic increase in ...",The increasing prevalence of metabolic syndrom...,Obesity reviews : an official journal of the I...,"Jos&#xe9; Ignacio Mart&#xed;nez-Montoro, Javie...",2023,,,,e13599,,10.1111/obr.13599,discussion
3,191,The burden of type 2 diabetes mellitus in stat...,Type 2 diabetes mellitus (T2D) is a complex me...,Type 2 diabetes mellitus (T2D) is a highly pre...,Obesity reviews : an official journal of the I...,"Carlos Alexandre Soares Andrade, Balqees Shahi...",2023,,,,e13593,,10.1111/obr.13593,discussion


## 6.3

In [35]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_table(table='sources', limit=len(text_df), order='DESC') # db_orm.py


    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!
Query: SELECT * from sources ORDER BY id DESC LIMIT 4
**Text #191 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5

In [36]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-07-21 08:23:46.152933-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,"In a recent study, researchers found that exer...",Discover the Best Time to Exercise for a Healt...,Did you know that the time you exercise can im...
1,2023-07-21 08:23:46.152933-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,"Hey, I just read some fascinating health resea...",Amazing Health Research Reveals Surprising Res...,Hey there! I just read this super cool study t...
2,2023-07-21 08:23:53.324742-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Make a more concise and simple version of t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey there! I just came across some fascinating...,Exciting Research on the Timing of Exercise an...,Check out this cool research I found! They dis...
3,2023-07-21 08:23:53.324742-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Make a more concise and simple version of t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,A recent study found that the timing of exerci...,Timing your exercise can improve metabolic health,Did you know that the time of day you exercise...
4,2023-07-21 08:24:00.558030-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey! I just read this super interesting resear...,Discover the Best Time to Exercise for Optimal...,Hey! Did you know that when you exercise can m...
5,2023-07-21 08:24:00.558030-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, th

## 6.31

In [37]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')


    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict


Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!
**Text #188 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 89 in C:\Use

In [38]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-07-21 08:26:51.877950-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey! I just read some really interesting resea...,New Research Reveals Surprising Health Benefit...,"Hey, guess what? I just found out that the tim..."
1,2023-07-21 08:26:51.877950-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey! I just came across some interesting resea...,Discover the Best Time to Exercise for a Healt...,Hey! Did you know that the time you exercise c...
2,2023-07-21 08:27:01.448126-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Make a more concise and simple version of t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey! I just came across some interesting resea...,Exercise timing affects metabolic health: Here...,Just found some cool research on exercise timi...
3,2023-07-21 08:27:01.448126-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Make a more concise and simple version of t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,"In recent health research, it was found that e...",New research shows that afternoon exercise may...,Did you know that exercising in the afternoon ...
4,2023-07-21 08:27:07.660828-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey! I just read an interesting research artic...,Exciting Health Research: Time of Exercise Mat...,Hey there! I just came across some really cool...
5,2023-07-21 08:27:07.660828-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, th

## 6.4

In [42]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')
    return sources_df

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!


In [43]:
qna_dict

,id,title,text,abstract,publication,authors,year,month,pub_volume,pub_issue,start_page,end_page,doi,section
0,188,A systematic review of patient barriers and fa...,Despite increasing awareness of the consequenc...,Numerous barriers are experienced by people wi...,Obesity reviews : an official journal of the I...,"Maxim de Jong, N&#xfa;ria Jansen, Marienke van...",2023,Aug,24,8,e13571,,10.1111/obr.13571,discussion
1,189,Dietary pulses as a means to improve the gut m...,"According to the World Health Organization, th...",A dysbiotic intestinal microbiome has been lin...,Obesity reviews : an official journal of the I...,"Hannah St John, &#xc9;ric Doucet, Krista A Power",2023,,,,e13598,,10.1111/obr.13598,discussion
2,190,Effects of exercise timing on metabolic health,"In the last decades, the dramatic increase in ...",The increasing prevalence of metabolic syndrom...,Obesity reviews : an official journal of the I...,"Jos&#xe9; Ignacio Mart&#xed;nez-Montoro, Javie...",2023,,,,e13599,,10.1111/obr.13599,discussion
3,191,The burden of type 2 diabetes mellitus in stat...,Type 2 diabetes mellitus (T2D) is a complex me...,Type 2 diabetes mellitus (T2D) is a highly pre...,Obesity reviews : an official journal of the I...,"Carlos Alexandre Soares Andrade, Balqees Shahi...",2023,,,,e13593,,10.1111/obr.13593,discussion


## 6.41

In [44]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!
**Text #188 prompt #1 of 4**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 89 in C:\Use

In [45]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-07-21 08:34:33.147173-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey there! I just read this fascinating resear...,Discover the Best Time to Exercise for a Healt...,Hey friend! Check out this cool research on wh...
1,2023-07-21 08:34:33.147173-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey there! I just came across some interesting...,New Research Reveals the Best Time to Exercise...,"Hey, guess what? I just found out that the tim..."
2,2023-07-21 08:34:40.919846-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Make a more concise and simple version of t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,"In recent research, it has been found that the...",Discover the Best Time to Exercise for Optimal...,Did you know that the time you exercise can af...
3,2023-07-21 08:34:40.919846-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Make a more concise and simple version of t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,"In the last decades, there has been a dramatic...",New research shows that timing of exercise can...,Did you know that the timing of exercise can i...
4,2023-07-21 08:34:46.852544-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey there! I came across some interesting rese...,Fascinating Research on Exercise Timing!,Hey! I just came across a mind-blowing study t...
5,2023-07-21 08:34:46.852544-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the drama

In [47]:
qna_dict['summary'].unique()

array(["Hey there! I just read this fascinating research on the best time to exercise for better health. Turns out, afternoon and evening exercise might be the way to go! 🌞💪🌙 The study showed that afternoon/evening exercise can help with glycemic control, blood pressure, and even lipid profile. Plus, it's especially effective for people with the evening chronotype.⏰🌃 But here's the fun part - morning exercise is great for weight loss! So, if you want to shed those pounds, a morning workout might be the way to go. 🏋️\u200d♀️🌅 Overall, exercise timing can have a big impact on our metabolism and well-being. Isn't that amazing? 😄🌟",
       "Hey there! I just came across some interesting research about exercise and metabolism that I think you'll find fascinating. According to the study, the timing of exercise can have a big impact on our metabolic health. It turns out that afternoon and evening exercise may be more effective than morning exercise when it comes to improving glycemic control,

## 6.5

In [1]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 1
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!
**Text #188 prompt #1 of 2**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 89 in C:\Use

## 6.51

In [2]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 2
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section)
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
	** Already exists in the database: The burden of type 2 diabetes mellitus in states of the European Union and United Kingdom at the national and subnational levels: A systematic review.
New records added successfully (if applicable)!
**Text #188 prompt #1 of 2**
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 89 in C:\Use

## 6.52

In [2]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full2'
section = 'discussion'
local = False
n_choices = 2
article_limit = 3
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1, 2, 3, 4])

Adding 3 rows to the database...
	** Already exists in the database: A systematic review of patient barriers and facilitators for implementing lifestyle interventions targeting weight loss in primary care.
	** Already exists in the database: Dietary pulses as a means to improve the gut microbiome, inflammation, and appetite control in obesity.
	** Already exists in the database: Effects of exercise timing on metabolic health.
New records added successfully (if applicable)!
**Text #188 prompt #1 of 2**
	Text ID: 188
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 89 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\orm_summarize.py : This model's maximum context length is 16385 tokens. However, your messages resulted 

In [3]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-07-21 08:48:40.043214-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey there! I just read some fascinating health...,Exciting Health Research - Exercise Timing,Hey there! I just read this awesome research a...
1,2023-07-21 08:48:40.043214-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey! I just read some interesting health resea...,Fascinating new health research!,Hey! Just read some amazing new health researc...
2,2023-07-21 08:48:46.590697-07:00,190,Effects of exercise timing on metabolic health,1,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey! I just came across some interesting resea...,Discover the surprising benefits of timing you...,Hey! Just stumbled upon this super cool info a...
3,2023-07-21 08:48:46.590697-07:00,190,Effects of exercise timing on metabolic health,2,"In the last decades, the dramatic increase in ...",You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full2,Hey there! I just read this really interesting...,Timing is everything: Exercise and metabolism,Hey there! Did you know that when you exercise...


## 6.6

In [4]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full'
section = 'discussion'
local = False
n_choices = 2
article_limit = 2
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

Adding 2 rows to the database...
	** Already exists in the database: A systematic review examining associations between physical activity, sedentary behaviour, and sleep duration with quality of life in older adults aged 65 years and above.
	** Already exists in the database: Advertising expenditures across media on food and beverage products heavily advertised on youth-appealing television stations in Canada.
New records added successfully (if applicable)!
**Text #174 prompt #1 of 2**
	Text ID: 174
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chaining class instance created
	Done creating prompt
	Sending request to gpt-3.5-turbo-16k-0613
		Requesting 2 choices using gpt-3.5-turbo-16k-0613
An error occurred on line 89 in C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src\orm_summarize.py : This model's maximum context length is 16385 tokens. However, you 

In [5]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-07-21 08:53:52.336866-07:00,175,Advertising expenditures across media on food ...,1,Unhealthy diets characterized by the consumpti...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Hey! I just read some interesting research on ...,Shocking Facts About Food Advertising and its ...,Hey! Guess what? I just found out some mind-bl...
1,2023-07-21 08:53:52.336866-07:00,175,Advertising expenditures across media on food ...,2,Unhealthy diets characterized by the consumpti...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Did you know that unhealthy diets filled with ...,Shocking Facts About Junk Food Advertising,Hey! Did you know that junk food advertising i...
2,2023-07-21 08:54:02.486978-07:00,175,Advertising expenditures across media on food ...,1,Unhealthy diets characterized by the consumpti...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Hey! You won't believe what I just read about ...,Junk food ads target teens and it's a big heal...,Hey there! Check out this cool study I found a...
3,2023-07-21 08:54:02.486978-07:00,175,Advertising expenditures across media on food ...,2,Unhealthy diets characterized by the consumpti...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Hey! Did you know that unhealthy diets and adv...,Shocking stats about unhealthy food ads!,Hey! Did you know that unhealthy diets and adv...


## 6.61

In [6]:
from functools import wraps
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\private")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from prompts import * # .py file stored in the path above
from db_orm import * 
from sources import *
from orm_summarize import *
from article_processing import *

@remote_sql_session
def get_from_queue(session, input_df, order_by='id', order='ASC'):
    """
    Return the matching records from the sources table as a pandas dataframe.

    Parameters:
    - input_df: A pandas DataFrame with the article records from the gpt_queue table or equivalent. Columns include 'title' and 'section'.
    - limit: The number of records to return.
    """
    def row_to_dict(row):
        result = session.query(Sources).filter_by(
            title=row['title'],
            section=row['section']
        ).limit(1).all()[0]
        
        sources_series = pd.Series({column.name: getattr(result, column.name) for column in result.__table__.columns})
        return sources_series

    sources_df = input_df.apply(row_to_dict, axis=1)
    ascending = True if order == 'ASC' else False
    sources_df.sort_values(order_by, ascending=ascending, inplace=True)
    return sources_df


#########
# Prep: Set parameters
folder_path = '../text/2023-07-14 full'
section = 'discussion'
local = False
n_choices = 1
article_limit = 4
temperature = 1
pause_per_request=0
# summary_iteration_id = iteration_id
iteration_id = 1
chatbot_id = iteration_id
model = 'gpt-3.5-turbo-16k-0613'
# model = 'gpt-4'
save_outputs=False

def generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local, article_limit=article_limit):
    ### Set up
    qna_dict = dict()
    chatbot_dict = dict()
    references_df_dict = dict()

    # set the option to wrap text within cells
    pd.set_option('display.max_colwidth', 50)
    pd.set_option('display.max_rows', 20)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    ####### 
    # Step 1: Create sources table
    if local:
        text_df = parse_fulltext(folder_path, section).iloc[:article_limit if article_limit else len(text_df)]
    else:
        text_df = get_table(table='gpt_queue', limit=article_limit) # db_orm.py
    references_df_dict[iteration_id] = create_sources_table(text_df) # sources.py

    ###### 
    # Step 2:  Add rows from gpt_queue table to sources table 
    bulk_append(table='sources', input_df=references_df_dict[iteration_id]) # db_orm.py

    # ##### 
    # Step 3: Get the new sources for summarization
    sources_df = get_from_queue(input_df=text_df, order_by='id', order='ASC')

    # ##### 
    # Step 4: Create summaries (functions contained in orm_summarize.py)
    chatbot_dict = batch_summarize( # orm_summarize.py
        sources_df, folder_path, prep_step, summarize_task, edit_task,  # parameter values found in prompts.py
        simplify_task, simplify_audience, format_task,
        chatbot_dict, temperature=temperature,
        system_role=system_role, model=model, max_tokens=1000,
        n_choices=n_choices, pause_per_request=pause_per_request,
        iteration_id=iteration_id, save_outputs=save_outputs
        )
    #########
    # Step 5: Create summaries table
    qna_dict = create_summaries_df(
        qna_dict, chatbot_dict, iteration_id, chatbot_id=chatbot_id
        )

    ##########
    # Step 5: Add results to summaries and prompts table 
    bulk_append(table='summaries', input_df=qna_dict[iteration_id]) # db_orm.py

    return qna_dict[iteration_id]

if __name__ == "__main__":
    qna_dict = generate_summaries(n_choices, temperature, model, pause_per_request, folder_path, section, local=True, article_limit=article_limit)
    # print(qna_dict[iteration_id)
    qna_dict



Keys for text_dict: dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

Adding 4 rows to the database...
	** Already exists in the database: A systematic review examining associations between physical activity, sedentary behaviour, and sleep duration with quality of life in older adults aged 65 years and above.
	** Already exists in the database: Advertising expenditures across media on food and beverage products heavily advertised on youth-appealing television stations in Canada.
	** Already exists in the database: Calcaneal tendon stiffness is not associated with dynamic time-dependent contractile output.
	** Already exists in the database: Creatine supplementation combined with blood flow restriction training enhances muscle thickness and performance: a randomized, placebo-controlled, and double-blind study.
New records added successfully (if applicable)!
**Text #174 prompt #1 of 2**
	Text ID: 174
Creating Chaining class instance
***OpenAI model: gpt-3.5-turbo-16k-0613
Chai

In [7]:
qna_dict

,timestamp,reference_id,article_title,choice,text,system_role,model,temperature,prep_step,summarize_task,edit_task,simplify_task,simplify_audience,format_task,full_summarize_task,folder,summary,headline,simple_summary
0,2023-07-21 08:55:50.359320-07:00,175,Advertising expenditures across media on food ...,1,Unhealthy diets characterized by the consumpti...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Did you know that unhealthy diets and food mar...,Shocking facts about food marketing to teenagers,Did you know that the food industry spends bil...
1,2023-07-21 08:55:56.337733-07:00,175,Advertising expenditures across media on food ...,1,Unhealthy diets characterized by the consumpti...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Hey friend! I just read a fascinating research...,Shocking Stats on Unhealthy Food Advertising t...,Hey buddy! Guess what I just found out? Unheal...
2,2023-07-21 08:56:06.644264-07:00,176,Calcaneal tendon stiffness is not associated w...,1,The rates at which muscles generate torque and...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Hey! I just came across a super interesting re...,Discover the Link Between Tendon Stiffness and...,"Hey! You know how our muscles work, right? Wel..."
3,2023-07-21 08:56:13.638023-07:00,176,Calcaneal tendon stiffness is not associated w...,1,The rates at which muscles generate torque and...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Hey! I just read a really interesting research...,Did you know? Tendon stiffness affects your mu...,Hey! Guess what? I just found out that the way...
4,2023-07-21 08:56:21.160177-07:00,177,Creatine supplementation combined with blood f...,1,Increases in muscular strength and hypertrophy...,You are someone who loves to read health resea...,gpt-3.5-turbo-16k-0613,1,"In the summary, cover the following informatio...",Write a casual text message to your friend abo...,Once you have written your text message: \...,3. Rewrite the text in a fun tone. Follow t...,people who like fun facts but don't know much ...,4. Return your final response in a JSON format...,Write a casual text message to your friend abo...,text/2023-07-14 full,Hey! I just read this incredible research abou...,Amazing Research about Muscle Strength and Cre...,Hey there! I just came across this awesome res...
5,2023-07-21 08:56:28.470882-07:00,177,Creatine supplementation combined with blood f...,1,Increases in muscular strength

# *End of Page*